Base에서 실행

# Textcuboid


## 1) Dataload Reuters news  data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.datasets import reuters

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(test_split=0.2)

category_list = []
for i in range(46):
    category_list.append(0)
print(len(category_list))

for i in train_labels:
    category_list[i] += 1

extracted_index = []

for i, v in enumerate(category_list):
    if v >= 100:
#         print(i, v)
        extracted_index.append(i)


extracted_train_data = []
for i, v in enumerate(train_labels):
    if v in extracted_index:
        extracted_train_data.append(train_data[i])

extracted_test_data = []
for i, v in enumerate(test_labels):
    if v in extracted_index:
        extracted_test_data.append(test_data[i])

extracted_train_labels = []
for i, v in enumerate(train_labels):
    if v in extracted_index:
        extracted_train_labels.append(train_labels[i])

extracted_test_labels = []
for i, v in enumerate(test_labels):
    if v in extracted_index:
        extracted_test_labels.append(test_labels[i])

word_to_index = reuters.get_word_index()

index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value+3] = key # pad, sos, unk, NULL 포함

for index, token in enumerate(("<pad>", "<sos>", "<unk>")): # index=3은 NULL
    index_to_word[index] = token

#문장 추출
to_txt=[]
for i in range(7828):
  
    to_txt.append(' '.join([index_to_word[index] for index in extracted_train_data[i]])[6:])
    
for i in range(1934):
    to_txt.append(' '.join([index_to_word[index] for index in extracted_test_data[i]])[6:])

46


In [3]:
y=extracted_train_labels+extracted_test_labels

encoder=LabelEncoder()

encoder.fit(y)

label=encoder.transform(y)

y_train=label[:7828]
y_test=label[7828:]

In [4]:
# 방법2) 케라스 내장함수 활용
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(y_train)
one_hot_test_labels = to_categorical(y_test)

In [5]:
#문서 리스트를 입력으로 문서가 설정한 길이(max_words)보다 단어 수가 많다면 뒤의 단어는 truncate하고 새로운 문서 리스트를 출력
def limit_words(text_list, max_words):
    new_list = []
    for text in text_list:
        words = text.split()
        if len(words) > max_words:
            words = words[:max_words]
        new_text = ' '.join(words)
        new_list.append(new_text)
    return new_list

In [6]:
to_txt=limit_words(to_txt,300)

In [7]:
to_txt

['mcgrath rentcorp said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3',
 "generale de banque sa lt genb br and lt heller overseas corp of chicago have each taken 50 pct stakes in factoring company sa belgo factors generale de banque said in a statement it gave no financial details of the transaction sa belgo factors' turnover in 1986 was 17 5 billion belgian francs reuter 3",
 'shr 3 28 dlrs vs 22 cts shr diluted 2 99 dlrs vs 22 cts net 46 0 mln vs 3 328 000 avg shrs 14 0 mln vs 15 2 mln year shr 5 41 dlrs vs 1 56 dlrs shr diluted 4 94 dlrs vs 1 50 dlrs net 78 2 mln vs 25 9 mln avg shrs 14 5 mln vs 15 1 mln note earnings per share reflect the two for one spli

In [8]:
#불용어 불러오기
import os
os.chdir('..')
with open('./english_stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = [line.strip() for line in file]

In [12]:
#필요 없는 단어 추가
num_lst=[str(i) for i in range(10000)]
stopwords=stopwords+['aaa', 'ab']+['00', '1990s', '1b', '2a','000','also','0','reuter','u','000', '03', '04', '09', '1985', '1986', '1987', '1988', '86',
       '87', '92', 'about','said','year','one','two','three','four','five','six','seven','eight','nine','ten','could','would',"'",'001', '002', '007050',
                                   '009', '01', '010', '019', '02', '0230', '025', '027', '029', '0300', '035', '037', '039', '040', '041', '042', '045',
                                   '05', '050', '054', '06', '006', '0600', '064', '065', '0667', '068', '07', '0700', '0730', '08', '0800', '0857', '0900',
                                   '0930', '097','033', '052']+num_lst

In [13]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(stop_words=stopwords)
X_dtm = vect.fit_transform(to_txt)
X_dtm = X_dtm.toarray()
X_new = SelectKBest(chi2, k=10000).fit(X_dtm, y)
TorF = X_new.get_support()
TorF
import numpy as np
word_view=np.array(vect.get_feature_names())
sw=word_view[TorF]

C:\Users\user\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
#선별된 10000개 단어
list(sw)

['0p',
 '10th',
 '11895',
 '12664',
 '16th',
 '17th',
 '18p',
 '18th',
 '1900gmt',
 '1950s',
 '1960s',
 '1970s',
 '1980s',
 '1a',
 '1p',
 '1st',
 '20s',
 '21950',
 '21p',
 '227p',
 '25th',
 '27095',
 '27758',
 '27p',
 '2nd',
 '2p',
 '2und',
 '32876',
 '32nd',
 '35000',
 '3a',
 '3p',
 '3rd',
 '40th',
 '459p',
 '48438',
 '4p',
 '4th',
 '50586',
 '51563',
 '51953',
 '56823',
 '58p',
 '595p',
 '5p',
 '630p',
 '63492',
 '634p',
 '643p',
 '78906',
 '79348',
 '80286',
 '80386',
 '82031',
 '84073',
 '86p',
 '95913',
 'aa',
 'aam',
 'abandon',
 'abandoned',
 'abastecimento',
 'abated',
 'abbas',
 'abdel',
 'abdelaziz',
 'abdul',
 'abdulla',
 'abdullah',
 'abe',
 'aberrational',
 'abide',
 'abiding',
 'abidjan',
 'abilene',
 'ability',
 'ablaze',
 'abnn',
 'aboard',
 'abolish',
 'abraham',
 'abrupt',
 'absence',
 'absent',
 'absolute',
 'absorb',
 'absorbs',
 'absurd',
 'abu',
 'abundant',
 'accelerated',
 'accelerating',
 'acceleration',
 'accent',
 'accentuating',
 'accept',
 'acceptance',
 'a

In [15]:
feature_lst10000=sw

## 2) textcuboid 생성

In [16]:
x_train=to_txt[:7828]
x_test=to_txt[7828:]

In [17]:
doc_lst=[]
for sen in x_train:
    doc_lst.append(sen.split())

In [18]:
test_lst=[]
for sen in x_test:
    test_lst.append(sen.split())

In [19]:
#Train_data에서 문서가 갖고 있는 선별한 feauture의 수 확인
count_lst=[]
for i in range(7828):
    total_feature_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in doc_lst[i]:
            total_feature_cnt+=1
    count_lst.append(total_feature_cnt)
    
print('Train_data에서 가장 많은 feature를 가진 문서의 경우 feature',max(count_lst),' 개를 가짐')
print('Train_data에서 가장 적은 feature를 가진 문서의 경우 feature',min(count_lst),' 개를 가짐')

Train_data에서 가장 많은 feature를 가진 문서의 경우 feature 119  개를 가짐
Train_data에서 가장 적은 feature를 가진 문서의 경우 feature 1  개를 가짐


In [20]:
#Test_data에서 문서가 갖고 있는 선별한 feauture의 수 확인
count_lst=[]
for i in range(1934):
    
    total_feature_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in test_lst[i]:
            total_feature_cnt+=1
    count_lst.append(total_feature_cnt)
    
print('Test_data에서 가장 많은 feature를 가진 문서의 경우 feature',max(count_lst),' 개를 가짐')
print('Test_data에서 가장 적은 feature를 가진 문서의 경우 feature',min(count_lst),' 개를 가짐')

Test_data에서 가장 많은 feature를 가진 문서의 경우 feature 111  개를 가짐
Test_data에서 가장 적은 feature를 가진 문서의 경우 feature 0  개를 가짐


In [21]:
#1-Channel TextCuboid 생성
textcuboid=[]

for i in range(7828):
    frame1=np.zeros((119,768))
    idx_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in doc_lst[i]:
            frame1[idx_cnt]=np.load('./bert_embedding/train(bert)/doc%d.npy'%i)[doc_lst[i].index(feature_lst10000[j])]
            idx_cnt+=1

    textcuboid.append(frame1)

textcuboid=np.array(textcuboid)

np.save('./1-Channel textcuboid_reuters(bert).npy',textcuboid)

In [22]:
#1-Channel TextCuboid 생성
textcuboid_test=[]

for i in range(1934):
    frame1=np.zeros((119,768))
    idx_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in test_lst[i]:
            frame1[idx_cnt]=np.load('./bert_embedding/test(bert)/test%d.npy'%i)[test_lst[i].index(feature_lst10000[j])]
            idx_cnt+=1

    textcuboid_test.append(frame1)

textcuboid_test=np.array(textcuboid_test)
np.save('./1-Channel textcuboid_test_reuters(bert).npy',textcuboid_test)